In [2]:
import cv2
import numpy as np
import random

# Take each frame
frame = cv2.imread('field-3369348_960_720.jpg')
facetmp = np.copy(frame)
# Convert BGR to HSV
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# define range of blue color in HSV
lower_blue = np.array([80,60,60])
upper_blue = np.array([130,255,255])

random.seed()

# Threshold the HSV image to get only blue colors
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Bitwise-AND mask and original image
res = cv2.bitwise_and(frame,frame, mask= mask)

In [3]:
mask2 = mask.sum(axis=(1)).astype(int)
t = mask2.max()
mask2[mask2 == t] = 0
MAX_Y = np.argmax(mask2)
MAX_X = mask.shape[1]
print(MAX_Y,MAX_X)

258 960


In [4]:
cv2.imshow('mask',mask)
cv2.imshow('res',res)
cv2.imshow('frame',frame)
cv2.waitKey()

-1

In [6]:
samolet = cv2.imread('plane2.png',-1)
plane = cv2.resize(samolet,None,fx=1/10, fy=1/10, interpolation = cv2.INTER_CUBIC)
cv2.imshow('plane',plane)
MAX_X = MAX_X - plane.shape[1]
cv2.waitKey()


-1

In [7]:
def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))
    #return face_part

In [8]:
def printImage2(face_img, overlay_t_img,startY,startX):
    result_2 = blend_transparent(facetmp[startX:startX + overlay_t_img.shape[0],startY:startY + overlay_t_img.shape[1]], overlay_t_img)
    facetmp[startX:startX + overlay_t_img.shape[0],startY:startY + overlay_t_img.shape[1]]= result_2
    #cv2.imshow('result',face_img[0:overlay_t_img.shape[0],0:overlay_t_img.shape[1]])

In [10]:
facetmp= np.copy(frame)
printImage2(frame,plane, random.randint(0, MAX_X),random.randint(0, MAX_Y))
cv2.imshow('image',facetmp)
cv2.waitKey()


-1

In [11]:
def draw_circle(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN:
        #facetmp= np.copy(frame)
        printImage2(frame,plane,random.randint(0, MAX_X),random.randint(0, MAX_Y))

In [12]:
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)


In [13]:
while(1):
    cv2.imshow('image',facetmp)
    k = cv2.waitKey(10)
    if k == 27:
        break